In [1]:
%autosave 1
%reload_ext autoreload
%autoreload 2
%load_ext line_profiler

Autosaving every 1 seconds


In [2]:
!pip install fastai
!pip install ipdb
!pip install opencv-python
!pip install nibabel
!pip install scikit-images

  ERROR: Could not find a version that satisfies the requirement scikit-images (from versions: none)
ERROR: No matching distribution found for scikit-images


In [10]:
from multiprocessing import Pool

import ipdb

import cv2
import nibabel as nib
from skimage import measure
from skimage import morphology

from fastai.vision import *
import datetime

### make/copy/link

In [11]:
dataset_dict = {
     'xn1': '/lung_general_data/Datasets/xinan/cov/CT20200206002',
#     'xn3':'/lung_general_data/Datasets/COV_DATA/xinan/CT20200217001',
#     'xn4':'/lung_general_data/Datasets/COV_DATA/xinan/CT20200220002',
    'xnb0': '/lung_general_data/Datasets/xinan/xnky_batch0',
#     'xnb1': '/lung_general_data/Datasets/xinan/xnky_batch1',
#     'xnnv': '/lung_general_data/Datasets/xinan/normal_virus/CT20200220003',
#     'xnb2':'/lung_general_data/Datasets/xinan/xnky_batch2/CT20200319003',
#     'hn1':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200225001',
#     'hn2':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200226001',
#     'hn3':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200226002',
#     'hn4':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200306001',
#     'hn5':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200308001',
#     'hn4t':'/lung_general_data/Datasets/COV_DATA/hndata/CT20200310003',
#     'gz1':'/lung_general_data/Datasets/COV_DATA/zzdata/CT20200219001'
}
dataset_label = {
    'xn1': '1',
#     'xn3':'1',
#     'xn4':'1',
#     'hn1':'1',
#     'hn2':'1',
#     'hn3':'1',
#     'hn4':'1',
#     'hn4t':'1',
#     'gz1':'1',
#     'hn5':'2',
    'xnb0':'2',
#     'xnb1':'2',
#     'xnb2':'2',
#     'xnnv':'2'
}

In [12]:
ds_root = Path('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia')

for ds_name,src in dataset_dict.items():
    sub_dir1 = ds_root/ds_name/'work_data/lung_det_xinan_keyan'
    sub_dir1.mkdir(parents=True, exist_ok=True)
    im_dir = sub_dir1/'3d_slice_origin'
    print(im_dir.exists())
    if not im_dir.exists():
        im_dir.symlink_to(Path(src)/'work_data/lung_det_xinan_keyan/3d_slice_origin')
    sub_dir2 = sub_dir1/'config/coco_json_origin'
    sub_dir2.mkdir(parents=True, exist_ok=True)
    src_fp = Path(src)/'work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt'
    #cmd = f'rsync -au i13:{src_fp} {sub_dir2}'
    cmd = f'rsync {src_fp} {sub_dir2}'
    print(cmd)
    os.system(cmd)

True
rsync /lung_general_data/Datasets/xinan/cov/CT20200206002/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt /lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/xn1/work_data/lung_det_xinan_keyan/config/coco_json_origin
True
rsync /lung_general_data/Datasets/xinan/xnky_batch0/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt /lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/xnb0/work_data/lung_det_xinan_keyan/config/coco_json_origin


In [16]:
cache_root = Path('/lung_general_data/jiechao/dataset0728')/'diag'


### prepare images

In [20]:
cache_root = Path('/lung_general_data/jiechao/dataset0728')/'diag'
(cache_root/'image').mkdir(parents=True, exist_ok=True)

ct_list = sorted(Path('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia').glob('*/work_data/lung_det_xinan_keyan/3d_slice_origin/*/*/*/'))
print(ct_list[:3])

#Need to rerun every time we add new data to the full-dataset
for ctid,ct_fp in enumerate(ct_list):
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    for k,png_fp in enumerate(png_list):
        # link image
        from_fp =cache_root/f'image/{ctid:06}_{k:04}.png'
        if not from_fp.exists():
            try:
                from_fp.symlink_to(f'{png_fp}')
            except:
                pass
            #from_fp.symlink_to(f'../../../{png_fp}')

[PosixPath('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/gz1/work_data/lung_det_xinan_keyan/3d_slice_origin/0825695/1.2.392.200036.9125.2.138612190166.20110508000138.20200127000094/1.2.156.14702.1.1001.17.1.2020012711122087315422238'), PosixPath('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/gz1/work_data/lung_det_xinan_keyan/3d_slice_origin/0825695/1.2.392.200036.9125.2.138612190166.20110508000138.20200127000094/1.2.156.14702.1.1001.17.1.2020012711122087320886872'), PosixPath('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/gz1/work_data/lung_det_xinan_keyan/3d_slice_origin/0825696/1.2.392.200036.9125.2.138612190166.20110508000138.20200127000157/1.2.156.14702.1.1001.17.1.2020012714534390611068001')]


### Make list for  CAP(0) vs COV(1) vs Others(0)

In [21]:
# find pids    
#MAKE POS 
all_pos_fps = {}
all_pos_pids = {}
all_fps = {}
all_pids = {}
print(dataset_dict.keys())
for subset in dataset_dict.keys():
    with open(f'/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/{subset}/work_data/lung_det_xinan_keyan/config/coco_json_origin/image_list_1221.txt','r') as f:
        all_pos_fps[subset] = sorted([l.strip() for l in f.readlines()])
        all_pos_pids[subset] = set([fp.split('/')[0] for fp in all_pos_fps[subset]])
    all_fps[subset] = sorted(Path(f'/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia/{subset}/work_data/lung_det_xinan_keyan/3d_slice_origin').glob('*/*/*/*.png'))
    all_pids[subset] = set([fp.parts[-4] for fp in all_fps[subset]])
    
#vl_pids = {k:np.random.choice(list(v), 10, replace=False).tolist() for k,v in all_pos_pids.items()}
# Choose 10% for validataion  
##### Run only once######
vl_pids = {subset : set(np.random.choice(list(v), int(0.1*len(v)), replace=False).tolist()) for subset , v in all_pos_pids.items()}
print(vl_pids)

dict_keys(['xn1', 'xnb0'])
{'xn1': {'0822184', '0822197', '0822199', '0822202', '0822220', '0822230', '0822182', '0822241', '0822183'}, 'xnb0': {'0804127', '0804110', '0804114'}}


In [37]:
# bg:0; cov:1; cap:2; 
ds_name = 'diag'

ct_list = sorted(Path('/lung_general_data/Datasets/lung_diagnosis/diag/data/lung_inflammation/origin/pneumonia').glob('*/work_data/lung_det_xinan_keyan/3d_slice_origin/*/*/*/'))

records = []
for ctid,ct_fp in enumerate(ct_list):
    if ct_fp.parts[9] == 'neg':
        break
    if ct_fp.parts[9] not in dataset_dict.keys():
        continue
    mask = dataset_label[ct_fp.parts[9]]
    image_from = Path(f'{ds_name}/image/{ctid:06}_')
    split_idx = 0
    for k,v in vl_pids.items():
        if ct_fp.parts[-3] in v:
            split_idx = 100
            break
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    h,w,d = None,None,len(png_list)
    for idx,png_fp in enumerate(png_list):
        if h is None:
            w,h = PIL.Image.open(png_fp).size
        if '/'.join(png_fp.parts[-4:]) in all_pos_fps[ct_fp.parts[9]]:
            mask += mask[0]
        else:
            mask += '0'
    records.append(['',mask,'',h,w,d,'det',image_from,'','',str(split_idx)])

In [38]:
len(records)

236

In [39]:
columns = ['image','mask','instance','h','w','d','src','image_from','mask_from','instance_from','split']

In [40]:
df = pd.DataFrame(records, columns=columns)
print([(df.split==str(i)).sum() for i in (0,100)])

df.loc[~((df.h==df.w)&(df.h==512)),'split'] = '300'
[(df.split==str(i)).sum() for i in (0,100)]

[211, 25]


[211, 25]

In [42]:
df.to_csv('/lung_general_data/jiechao/dataset0728/dd'+ str(datetime.datetime.now()) + '.csv', index=False)

In [43]:
df.tail()

,image,mask,instance,h,w,d,src,image_from,mask_from,instance_from,split
231,,2000000000022222222222222222222222222222222222...,,512,512,110,det,diag/image/008385_,,,0
232,,2000000000022222222222222222222222222222222222...,,512,512,67,det,diag/image/008386_,,,0
233,,2000000000000000000002222222222222222222222222...,,512,512,97,det,diag/image/008387_,,,0
234,,2000000220222222222222222222222222222222222222...,,512,512,83,det,diag/image/008388_,,,0
235,,2000000000022222222222222222222222222222222222...,,512,512,88,det,diag/image/008389_,,,0


In [44]:
#统计所有样本的数量（包含了0）
df.d.sum()

43483

In [45]:
# 统计正负样本的CT数量
for i in range(0,3):
    print(sum([t[0]==f'{i}' for t in df['mask']]))

0
199
37


### make list hainan(by maoli)

In [161]:
# xlsx to csv
tdf = pd.read_excel('origin/hainan/gt.xlsx')
tdf.to_csv('origin/hainan/gt.csv', index=False)

In [167]:
cache_root = Path('a1')/'diag_hainan'
(cache_root/'image').mkdir(parents=True)

ct_list = sorted(Path('origin/hainan/png').glob('*/'))

for ctid,ct_fp in enumerate(ct_list):
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    for k,png_fp in enumerate(png_list):
        # link image
        from_fp =cache_root/f'image/{ctid:06}_{k:04}.png'
        if not from_fp.exists():
            from_fp.symlink_to(f'../../../{png_fp}')

In [179]:
# now: cov:1; non-co-virus:0
#  to: bg:0; cov:1; bacterial:2; fungal:3; non-co-virus:4

tdf = pd.read_csv('origin/hainan/gt.csv')

records = []
for ctid,ct_fp in enumerate(ct_list):
    pid,date = ct_fp.name.split('_')
    image_from = Path(f'diag_hainan/image/{ctid:06}_')
    png_list = [t for t in sorted(list(ct_fp.glob('*.png'))) if '-' not in str(t.stem)]
    d = len(png_list)
    w,h = PIL.Image.open(png_list[0]).size
    found = (tdf.pid==int(pid))&(tdf.date==int(date))
    if found.sum()>0:
        label = tdf.loc[found].label.values[0]
    else:
        assert ct_fp.name=='0826166_20200127'
        label = 1
    if label==0:
        label = 4
    mask = f'{label}' * (d+1)
    records.append(['',mask,'',h,w,d,'hainan',image_from,'','','0'])

In [180]:
columns = ['image','mask','instance','h','w','d','src','image_from','mask_from','instance_from','split']

In [181]:
df = pd.DataFrame(records, columns=columns)
df.to_csv('dd_hainan.csv', index=False)

In [178]:
df.head()

,image,mask,instance,h,w,d,src,image_from,mask_from,instance_from,split
0,,1111111111111111111111111111111111111111111111...,,512,512,201,hainan,diag_hainan/image/000000_,,,0
1,,1111111111111111111111111111111111111111111111...,,512,512,207,hainan,diag_hainan/image/000001_,,,0
2,,1111111111111111111111111111111111111111111111...,,512,512,209,hainan,diag_hainan/image/000002_,,,0
3,,1111111111111111111111111111111111111111111111...,,512,512,217,hainan,diag_hainan/image/000003_,,,0
4,,1111111111111111111111111111111111111111111111...,,512,512,184,hainan,diag_hainan/image/000004_,,,0


#### Negative Slices

In [ ]:
ds_root = Path('origin/pneumonia')
im_dir = ds_root/'neg/work_data/lung_det_xinan_keyan/3d_slice_origin'
im_dir.mkdir(parents=True, exist_ok=True)
list_dir = ds_root/'neg/work_data/lung_det_xinan_keyan/config/coco_json_origin'
list_dir.mkdir(parents=True, exist_ok=True)

#
with open(list_dir/'nodule_lesion_random_0611_train_image_list.txt','r') as f:
    fp_list = [l.strip() for l in f.readlines()]
    
fp_list[:2]

#Symlink image in neglist 
src_dir = Path('/lung_general_data/lung_det/lung_nodule_data/lung_nodule_0213/3d_slice_origin')
dst_dir = im_dir
#dst_dir.symlink_to(src_dir)

for fp in fp_list:
    fp = Path(fp)
    i = int(fp.stem)
    for j in range(i-5,i+6):
        if i<0:
            continue
        im_fp = fp.parent/f'{j:03}.png'
        dst_fp = dst_dir/im_fp
        dst_fp.parent.mkdir(parents=True, exist_ok=True)
        #print(src_dir/dst_fp)
        #cmd = f'rsync -au i13:{src_dir/im_fp} {dst_fp}'
        dst_fp.symlink_to(src_dir/im_fp)
        #print(cmd)
        #os.system(cmd)